In [5]:
import numpy as np
from numpy import sqrt
import torch
import tntorch as tn
import matplotlib.pyplot as plt
from qiskit.circuit.library import UnitaryGate

%matplotlib inline
num_dimensions = 4
mu = np.array([0.10, 0.10, 0.23, 0.17])  # Mean vector
cov_matrix = np.array([[0.20, 0.35, 0.12, 0.23],[0.10, 0.28, 0.19, 0.13],[0.10, 0.20, 0.10, 0.10],[0.19, 0.03, 0.07, 0.27]])  # Covariance matrix 
cov_matrix = 0.5 * (cov_matrix + cov_matrix.T)  
cov_matrix = cov_matrix @ cov_matrix

scale = 50
cov_matrix = scale * cov_matrix

cov_matrix_inv = np.linalg.inv(cov_matrix)


det = np.linalg.det(cov_matrix)
def gaussian(x):
    return 1/ (((2 * np.pi) ** (num_dimensions) * np.abs(det)) ** 0.5) * np.exp(-0.5 * ((x-mu).T @ cov_matrix_inv @ (x-mu)))


#Discretization parameter
d = 12                # qubit
m = 2**(d//4)         # physical qubits for each dimension

print(m)

vectroized_function = []
for x in np.linspace(mu[0] - np.sqrt(cov_matrix[0,0]), mu[0] + np.sqrt(cov_matrix[0,0]), m):
    for y in np.linspace(mu[1] - np.sqrt(cov_matrix[1,1]), mu[1] + np.sqrt(cov_matrix[1,1]), m):
        for z in np.linspace(mu[2] - np.sqrt(cov_matrix[2,2]), mu[2] + np.sqrt(cov_matrix[2,2]), m):
            for w in np.linspace(mu[3] - np.sqrt(cov_matrix[3,3]), mu[3] + np.sqrt(cov_matrix[3,3]), m):
                vectroized_function.append(gaussian(np.array([x,y,z,w])))

vectroized_function = np.array(vectroized_function)

shape = (2,)*d                         # (2,2,2,2)
A = vectroized_function.reshape(shape) # numpy tensor
T=tn.Tensor(A)                         #tensore torch


TTrain = tn.cross(
    function=lambda x: x,
    tensors=[T],            
    ranks_tt=64,                 
)


print(TTrain)

cores_torch = TTrain.cores
cores = [c.cpu().numpy() for c in cores_torch]

8
cross device is cpu
Functions that require cross-approximation can be accelerated with the optional maxvolpy package, which can be installed by 'pip install maxvolpy'. More info is available at https://bitbucket.org/muxas/maxvolpy.
Cross-approximation over a 12D domain containing 4096 grid points:
iter: 0  | eps: 8.293e-16 | time:   0.1115 | largest rank:  64 <- converged: eps < 1e-06
Did 21836 function evaluations, which took 1.24e-05s (1.761e+09 evals/s)

12D TT tensor:

  2   2   2   2   2   2   2   2   2   2   2   2
  |   |   |   |   |   |   |   |   |   |   |   |
 (0) (1) (2) (3) (4) (5) (6) (7) (8) (9) (10)(11)
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
1   2   4   8   16  32  64  32  16  8   4   2   1



In [6]:
import numpy as np

def contract_tt_cores(cores):
    """
    Contraction of the TT cores.
    Input:
      numpy array of shape (r_{k-1}, i_k, r_k)
    Output:
      numpy array of shape (i_1, i_2, ..., i_d)
    """
    result = cores[0]

    for C in cores[1:]:
        result = np.tensordot(result, C, axes=([-1], [0]))
        # we go from (..., i_k, r_k) ⨂ (r_k, i_{k+1}, r_{k+1})
        # to → (..., i_k, i_{k+1}, r_{k+1})
    
    # Now we remove trivial dimension
    return np.squeeze(result)


full_tensor = contract_tt_cores(cores)

print("Reconstructed shape:", full_tensor.shape)
# Should print (2,2,2,2,2,2,2,2,2,2) for d=10

#reshape the tensor
Z = full_tensor.reshape((m, m, m, m))

Reconstructed shape: (2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)


In [7]:
def gaussian_4d(x_vals, y_vals, z_vals, w_vals, mu, cov_matrix):
    """
    Calcola la densità di una gaussiana multivariata 4D su una griglia definita
    dai vettori x_vals, y_vals, z_vals, w_vals.

    Parametri:
    - x_vals, y_vals, z_vals, w_vals: 1D array di coordinate lungo ciascuna dimensione
    - mu: vettore di media di dimensione 4
    - cov_matrix: matrice di covarianza 4x4

    Ritorna:
    - F: array 4D con shape (len(x_vals), len(y_vals), len(z_vals), len(w_vals))
    """
    # Invertere la covarianza e calcolare il fattore di normalizzazione
    inv_cov = np.linalg.inv(cov_matrix)
    det_cov = np.linalg.det(cov_matrix)
    norm_const = 1 / (((2 * np.pi) ** 4 * det_cov) ** 0.5)

    # Preallocazione
    F = np.zeros((len(x_vals), len(y_vals), len(z_vals), len(w_vals)))

    # Cicli annidati per tutti i punti della griglia
    for i, xv in enumerate(x_vals):
        for j, yv in enumerate(y_vals):
            for k, zv in enumerate(z_vals):
                for l, wv in enumerate(w_vals):
                    x = np.array([xv, yv, zv, wv])
                    diff = x - mu
                    exponent = -0.5 * diff.T @ inv_cov @ diff
                    F[i, j, k, l] = norm_const * np.exp(exponent)

    return F

In [8]:
x_vals = np.linspace(mu[0] - np.sqrt(cov_matrix[0,0]), mu[0] + np.sqrt(cov_matrix[0,0]), m)
y_vals = np.linspace(mu[1] - np.sqrt(cov_matrix[1,1]), mu[1] + np.sqrt(cov_matrix[1,1]), m)
z_vals = np.linspace(mu[2] - np.sqrt(cov_matrix[2,2]), mu[2] + np.sqrt(cov_matrix[2,2]), m)
w_vals = np.linspace(mu[3] - np.sqrt(cov_matrix[3,3]), mu[3] + np.sqrt(cov_matrix[3,3]), m)
dx = x_vals[1] - x_vals[0]
dy = y_vals[1] - y_vals[0]
dz = z_vals[1] - z_vals[0]
dw = z_vals[1] - z_vals[0]

integrale_num = np.sum(np.abs(Z)) * dx * dy * dz * d
distribuzione = gaussian_4d(x_vals, y_vals, z_vals, w_vals, mu, cov_matrix)

F = np.zeros((len(x_vals),len(x_vals),len(x_vals),len(x_vals)))
G = np.zeros((len(x_vals),len(x_vals),len(x_vals),len(x_vals)))

for x in range(len(x_vals)):
    for y in range(len(x_vals)):
        for z in range(len(x_vals)):
            for w in range(len(x_vals)):
                F = np.sum(distribuzione[:x,:y,:z,:w])
                G = np.sum(np.abs(Z[:x,:y,:z,:w]))

# Maximum absolute difference
D = np.max(np.abs(F - G))
print(D)

1.7763568394002505e-15
